In [20]:
#import modules:
import pandas as pd
#import numpy as np
import glob, os, time

In [21]:
#time the process
start = time.time()

In [22]:
#define variables - update these each month:
inpath = (r'C:\Users\kellyjo00\Box\FCCS Reporting\Jul-20\CSV TBs')
outpath = (r'C:\Users\kellyjo00\Box\FCCS Reporting\Jul-20\Python TBs')
all_filenames = [i for i in glob.glob(os.path.join(inpath,"*.csv"))]
all_filenames

['C:\\Users\\kellyjo00\\Box\\FCCS Reporting\\Jul-20\\CSV TBs\\20200630_TB_Contribution_Pivot_EUR_FY19ALL-20200716-1255.csv',
 'C:\\Users\\kellyjo00\\Box\\FCCS Reporting\\Jul-20\\CSV TBs\\20200630_TB_Contribution_Pivot_EUR_FY19_P0-20200716-1307.csv',
 'C:\\Users\\kellyjo00\\Box\\FCCS Reporting\\Jul-20\\CSV TBs\\20200713_TB_Contribution_Pivot_EUR_FY20_P0-20200716-1308.csv',
 'C:\\Users\\kellyjo00\\Box\\FCCS Reporting\\Jul-20\\CSV TBs\\20200713_TB_Contribution_Pivot_EUR_FY20_P1-P6-20200716-1255.csv']

In [23]:
#read csv files and create initial DataFrame
df = pd.concat([pd.read_csv(f, 
                            header=0,
                            engine='c',
                            usecols=[0,1,2,5,7,8,9,10,11,12,13,14,17], 
                            names=['Years', 'Period','Entity','Account','Movement','Intercompany','Data Source','Project Code','Cost Center','Ref Code','Product Line','Functional Currency','3 Contribution EUR'],
                            na_values=['-']
                           ) for f in all_filenames], ignore_index=True)
df.shape

(519906, 13)

In [24]:
# convert all '3 Contribution EUR' values to nunmeric - "errors='coerce'" converts non-numeric values to NaN
try:
    cols = ['3 Contribution EUR']
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
except KeyError:
    print("No non-numeric values found")
df.shape

(519906, 13)

In [25]:
#drop NaN values from '3 Contribution EUR'
df = df.dropna(axis='rows')
df.shape

(519906, 13)

In [26]:
#add 'Value' Column in order to create pivot table properly
df['Value']='Balance'
df.shape

(519906, 14)

In [27]:
#pivot the data in the previously created DataFrame by year and period (Consol DF)
consol_df = df.pivot_table(
    values='3 Contribution EUR', 
    index=['Years','Entity','Account','Movement','Intercompany','Data Source','Project Code','Cost Center','Ref Code','Product Line','Functional Currency'],
    columns=['Value','Period'],
    aggfunc='sum', fill_value=0)
consol_df.shape
#consol_df.to_csv(os.path.join(outpath,"pythonOutput-" + time.strftime("%Y%m%d-%H%M") + ".csv"), index=True, encoding='utf-8-sig')

(77618, 13)

In [28]:
#add the calculated movement columns
consol_df['Movement','P1'] = (consol_df['Balance','P1'] - consol_df['Balance','P0'])
consol_df['Movement','P2'] = (consol_df['Balance','P2'] - consol_df['Balance','P1'])
consol_df['Movement','P3'] = (consol_df['Balance','P3'] - consol_df['Balance','P2'])
consol_df['Movement','P4'] = (consol_df['Balance','P4'] - consol_df['Balance','P3'])
consol_df['Movement','P5'] = (consol_df['Balance','P5'] - consol_df['Balance','P4'])
consol_df['Movement','P6'] = (consol_df['Balance','P6'] - consol_df['Balance','P5'])
consol_df['Movement','P7'] = (consol_df['Balance','P7'] - consol_df['Balance','P6'])
consol_df['Movement','P8'] = (consol_df['Balance','P8'] - consol_df['Balance','P7'])
consol_df['Movement','P9'] = (consol_df['Balance','P9'] - consol_df['Balance','P8'])
consol_df['Movement','P10'] = (consol_df['Balance','P10'] - consol_df['Balance','P9'])
consol_df['Movement','P11'] = (consol_df['Balance','P11'] - consol_df['Balance','P10'])
consol_df['Movement','P12'] = (consol_df['Balance','P12'] - consol_df['Balance','P11'])

consol_df.shape

(77618, 25)

In [29]:
#drop P0 from Balances
consol_df.drop('P0', axis=1, level=1, inplace=True)
consol_df.shape

(77618, 24)

In [30]:
#stack Periods
stacked_df = consol_df.stack(1)
stacked_df.shape

(931416, 2)

In [31]:
#export to csv
stacked_df.to_csv(os.path.join(outpath,"pythonOutput-" + time.strftime("%Y%m%d-%H%M") + ".csv"), index=True, encoding='utf-8-sig')

In [32]:
end = time.time()
print(end - start)

62.31866455078125
